# Emissions mitigation widget  
Data preparation for the emissions mitigation widget.  
Data model is as follows: 
  
`indicator`: 'Reduce deforestation', 'Reforestation (Tropics)', 'Forest Management (Global)', 'Grassland and savanna fire mgmt', 'Reduce peatland degradation and conversion', 'Reduce mangrove loss', 'Mangrove Restoration'  
`category`: "Mangrove" or "Other"  
`value`: _int_    
`year`: 2020



In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona

In [15]:
df = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/emissions_per_hectare_20221213.csv')
df.head()

,ISO,Country,defor_techden,refor_techden,formgmts_techden,gfire_techden,peat_techden,mangrove_techden,mangrest_techden,mangrest_feas,...,manure_techcum,manure_tech_animal,manure_techint,manure_feas,manure_feascum,manure_feas_animal,manure_feasint,riceg_tech,riceg_techcum,riceg_tech_ha
0,AGO,Angola,219.6426,179.0867,4.1586,18.1093,NaN,1404.4666,402.323800,0.0041,...,0.672,158767.058,0.139,0.012,0.352,64983.169,0.178,0.00,0.0,57160.0
1,AIA,Anguilla,136.5638,NaN,0.0000,NaN,NaN,1215.9538,300.931300,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATG,Antigua & Barbuda,136.5638,NaN,0.0000,NaN,NaN,1342.5897,287.929600,0.0001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABW,Aruba,136.5638,NaN,0.0000,NaN,NaN,1268.6633,235.318549,0.0001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUS,Australia,NaN,238.2483,17.0699,5.6156,1263.144,1280.0382,206.181084,0.2368,...,32.801,4013247.877,0.270,0.322,9.710,900790.321,0.357,0.28,8.4,82204.0


In [16]:
columns_to_use = ['ISO','defor_techden', 'refor_techden', 'formgmts_techden',
                    'gfire_techden', 'peat_techden', 'mangrove_techden',
                    'mangrest_techden']
df = df[columns_to_use].copy()
df.head(2)

,ISO,defor_techden,refor_techden,formgmts_techden,gfire_techden,peat_techden,mangrove_techden,mangrest_techden
0,AGO,219.6426,179.0867,4.1586,18.1093,NaN,1404.4666,402.3238
1,AIA,136.5638,NaN,0.0000,NaN,NaN,1215.9538,300.9313


In [17]:
df_long = pd.melt(df, id_vars='ISO')
df_long

,ISO,variable,value
0,AGO,defor_techden,219.6426
1,AIA,defor_techden,136.5638
2,ATG,defor_techden,136.5638
3,ABW,defor_techden,136.5638
4,AUS,defor_techden,NaN
...,...,...,...
737,VNM,mangrest_techden,667.8428
738,VIR,mangrest_techden,288.6800
739,YEM,mangrest_techden,182.8482
740,ZMB,mangrest_techden,NaN


In [18]:
df_long.variable.unique()

array(['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'], dtype=object)

In [19]:
df_vars = pd.DataFrame(data={'variable':['defor_techden', 'refor_techden', 'formgmts_techden',
       'gfire_techden', 'peat_techden', 'mangrove_techden',
       'mangrest_techden'],
       'indicator':['Reduce deforestation','Reforestation (Tropics)','Forest Management (Global)','Grassland and savanna fire mgmt',
       'Reduce peatland degradation and conversion','Reduce mangrove loss','Mangrove Restoration'],
       'category': ['Other','Other','Other','Other','Other','Mangrove','Mangrove']})
df_vars

,variable,indicator,category
0,defor_techden,Reduce deforestation,Other
1,refor_techden,Reforestation (Tropics),Other
2,formgmts_techden,Forest Management (Global),Other
3,gfire_techden,Grassland and savanna fire mgmt,Other
4,peat_techden,Reduce peatland degradation and conversion,Other
5,mangrove_techden,Reduce mangrove loss,Mangrove
6,mangrest_techden,Mangrove Restoration,Mangrove


In [20]:
df_long = pd.merge(df_long, df_vars, on = 'variable')
df_long.rename(columns={'ISO': 'iso'}, inplace=True)
df_long

,iso,variable,value,indicator,category
0,AGO,defor_techden,219.6426,Reduce deforestation,Other
1,AIA,defor_techden,136.5638,Reduce deforestation,Other
2,ATG,defor_techden,136.5638,Reduce deforestation,Other
3,ABW,defor_techden,136.5638,Reduce deforestation,Other
4,AUS,defor_techden,NaN,Reduce deforestation,Other
...,...,...,...,...,...
737,VNM,mangrest_techden,667.8428,Mangrove Restoration,Mangrove
738,VIR,mangrest_techden,288.6800,Mangrove Restoration,Mangrove
739,YEM,mangrest_techden,182.8482,Mangrove Restoration,Mangrove
740,ZMB,mangrest_techden,NaN,Mangrove Restoration,Mangrove


## Add location ids

In [11]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country'][['iso', 'location_idn']]
locations.head()

,iso,location_idn
82,QAT,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd
89,MYT,0750953f-4af9-549b-aeea-329663249a56
118,VNM,09a1ab14-11ad-56ec-8acb-a149e5697abd
132,GRD,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e
149,IND,0c07ca53-7b17-5650-a2c6-0cc27249a4bd


In [12]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [36]:
#Add ISO codes
df_final = pd.merge(df_long, locations, on = 'iso', how='left')
df_final = pd.merge(df_final, api_locs, on = 'location_idn')
df_final.head()

,iso,variable,value,indicator,category,location_idn,id
0,AGO,defor_techden,219.6426,Reduce deforestation,Other,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
1,AGO,refor_techden,179.0867,Reforestation (Tropics),Other,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
2,AGO,formgmts_techden,4.1586,Forest Management (Global),Other,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
3,AGO,gfire_techden,18.1093,Grassland and savanna fire mgmt,Other,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029
4,AGO,peat_techden,NaN,Reduce peatland degradation and conversion,Other,27ceab8c-946e-5286-a06f-8bd98ec81f77,2029


## Final format

In [37]:
df_final.drop(columns=['iso','variable','location_idn'], inplace=True)
#df_final['unit'] = 'tCO2eq'

df_final.rename(columns={'variable': 'indicator', 'id': 'location_id'}, inplace=True)
df_final = df_final[df_final['value'].notnull()]
df_final

,value,indicator,category,location_id
0,219.6426,Reduce deforestation,Other,2029
1,179.0867,Reforestation (Tropics),Other,2029
2,4.1586,Forest Management (Global),Other,2029
3,18.1093,Grassland and savanna fire mgmt,Other,2029
5,1404.4666,Reduce mangrove loss,Mangrove,2029
...,...,...,...,...
713,288.6800,Mangrove Restoration,Mangrove,2339
714,83.7280,Reduce deforestation,Other,2690
715,146.8523,Reforestation (Tropics),Other,2690
719,996.1105,Reduce mangrove loss,Mangrove,2690


In [30]:
df_final.to_csv('../../../../data/mitigation_emission_data_202212.csv', index=False)

In [32]:
df_long[df_long.iso == 'COL']

,iso,variable,value,indicator,category
18,COL,defor_techden,376.2110,Reduce deforestation,Other
124,COL,refor_techden,243.8546,Reforestation (Tropics),Other
230,COL,formgmts_techden,6.4919,Forest Management (Global),Other
336,COL,gfire_techden,4.2426,Grassland and savanna fire mgmt,Other
442,COL,peat_techden,1183.9756,Reduce peatland degradation and conversion,Other
548,COL,mangrove_techden,1985.1214,Reduce mangrove loss,Mangrove
654,COL,mangrest_techden,404.6449,Mangrove Restoration,Mangrove
